In [70]:
import keras
import tensorflow as tf
from keras.layers import Conv2D
#import keras.activations as Activation
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.core import Dropout
from keras.layers import Add
from keras.layers import Concatenate

from keras.models import Model
from keras.models import Sequential
from keras.layers import Input


In [88]:
input_shape = (448,448,3)

net = {}
# input block
input_tensor = Input(shape=input_shape)
net['input'] = input_tensor
# block 1
'''
input size: (1,3,448,448)
output size: (1,64,224,224)
'''
net['conv1_1'] = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_1')(net['input'])
net['conv1_2'] = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(net['conv1_1'])
net['pool1'] = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(net['conv1_2'])

# block 2
''' output size: (1,128,112,112)'''
net['conv2_1'] = Conv2D(128, (3,3), activation='relu', padding='same', name='conv2_1')(net['pool1'])
net['conv2_2'] = Conv2D(128, (3,3), activation='relu', padding='same', name='conv2_2')(net['conv2_1'])
net['pool2'] = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(net['conv2_2'])

# block 3
''' output size: (1,256,112,112)'''
net['conv3_1'] = Conv2D(256, (3,3), activation='relu', padding='same', name='conv3_1')(net['pool2'])
net['conv3_2'] = Conv2D(256, (3,3), activation='relu', padding='same', name='conv3_2')(net['conv3_1'])
net['conv3_3'] = Conv2D(256, (3,3), activation='relu', padding='same', name='conv3_3')(net['conv3_2'])
net['pool3'] = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(net['conv3_3'])

# block 4
''' 
output size: (1,512,56,56)
original pool4 is removed (or use a element wise pool)
'''
net['conv4_1'] = Conv2D(512, (3,3), activation='relu', padding='same', name='conv4_1')(net['pool3'])
net['conv4_2'] = Conv2D(512, (3,3), activation='relu', padding='same', name='conv4_2')(net['conv4_1'])
net['conv4_3'] = Conv2D(512, (3,3), activation='relu', padding='same', name='conv4_3')(net['conv4_2']) 

# block 5
''' 
output size: (1,512,56,56)
original pool5 is removed (or use a element wise pool)
'''
net['conv5_1'] = Conv2D(512, (3,3), activation='relu', padding='same', name='conv5_1')(net['conv4_3'])
net['conv5_2'] = Conv2D(512, (3,3), activation='relu', padding='same', name='conv5_2')(net['conv5_1'])
net['conv5_3'] = Conv2D(512, (3,3), activation='relu', padding='same', name='conv5_3')(net['conv5_2'])

# block 6 - FCN BN and Dropout
weight_filter = keras.initializers.RandomNormal(mean=0.0, stddev=0.001)
'''how to set padding value in keras??? should be 3'''
net['fcn6'] = Conv2D(1024, (7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fcn6')(net['conv5_3'])
net['fconv6_BN'] = BatchNormalization(center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')(net['fcn6'])
net['fconv6_BN'] = Activation('relu')(net['fconv6_BN'])
net['fconv6_BN'] = Dropout(rate=0.5)(net['fconv6_BN'])


[<keras.engine.topology.InputLayer object at 0x7ff4e51a4860>, <keras.layers.convolutional.Conv2D object at 0x7ff4e51a4898>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5203ba8>, <keras.layers.pooling.MaxPooling2D object at 0x7ff4e51a4978>, <keras.layers.convolutional.Conv2D object at 0x7ff4e514e2e8>, <keras.layers.convolutional.Conv2D object at 0x7ff4e51c4c18>, <keras.layers.pooling.MaxPooling2D object at 0x7ff4e514e9b0>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5182be0>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5171c18>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5113f28>, <keras.layers.pooling.MaxPooling2D object at 0x7ff4e5122588>, <keras.layers.convolutional.Conv2D object at 0x7ff4e50e4710>, <keras.layers.convolutional.Conv2D object at 0x7ff4e51458d0>, <keras.layers.convolutional.Conv2D object at 0x7ff4e50e4550>, <keras.layers.convolutional.Conv2D object at 0x7ff4e50a9ac8>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5087eb8>, <keras.l

In [102]:
# ----------------------------------------- Structure 3-2-1 ---------------------------------------

# fconv7_3, padding should be 3
weight_filter = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
net['fconv7_3']=Conv2D(128, (1,1), activation='relu', padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_3')(net['fconv6_BN'])
net['fconv7_3to2_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_3to2_step1')(net['fconv7_3'])
net['fconv7_3to2_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_3to2_step2')(net['fconv7_3to2_step1'])

# fconv7_2, padding should be 3
net['fconv7_2']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_2')(net['fconv6_BN'])
net['fconv7_2+']=Add()([net['fconv7_3to2_step2'],net['fconv7_2']]) # element-wise sum
net['fconv7_2+']=Activation('relu')(net['fconv7_2+'])
net['fconv7_2to1_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_2to1_step1')(net['fconv7_2+'])
net['fconv7_2to1_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_2to1_step2')(net['fconv7_2to1_step1'])

# ----------------------------------------- Structure 6-5-4 ---------------------------------------

# fconv7_6, padding should be 3
weight_filter = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
net['fconv7_6']=Conv2D(128, (1,1), activation='relu', padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_6')(net['fconv6_BN'])
net['fconv7_6to5_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_6to5_step1')(net['fconv7_6'])
net['fconv7_6to5_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_6to5_step2')(net['fconv7_6to5_step1'])

# fconv7_5, padding should be 3
net['fconv7_5']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_5')(net['fconv6_BN'])
net['fconv7_5+']=Add()([net['fconv7_6to5_step2'],net['fconv7_5']]) # element-wise sum
net['fconv7_5+']=Activation('relu')(net['fconv7_5+'])
net['fconv7_5to4_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_5to4_step1')(net['fconv7_5+'])
net['fconv7_5to4_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_5to4_step2')(net['fconv7_5to4_step1'])

print (net['fconv7_2to1_step2'])
print (net['fconv7_5to4_step2'])

# ----------------------------------------- Structure 9-8-7-1 ------------------------------------------------

# fconv7_9, padding should be 3
weight_filter = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
net['fconv7_9']=Conv2D(128, (1,1), activation='relu', padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_9')(net['fconv6_BN'])
net['fconv7_9to8_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_9to8_step1')(net['fconv7_9'])
net['fconv7_9to8_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_9to8_step2')(net['fconv7_9to8_step1'])

# fconv7_8, padding should be 3
net['fconv7_8']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_8')(net['fconv6_BN'])
net['fconv7_8+']=Add()([net['fconv7_9to8_step2'],net['fconv7_8']]) # element-wise sum
net['fconv7_8+']=Activation('relu')(net['fconv7_8+'])
net['fconv7_8to7_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_8to7_step1')(net['fconv7_8+'])
net['fconv7_8to7_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_8to7_step2')(net['fconv7_8to7_step1'])

# fconv7_7, padding should be 3
net['fconv7_7']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_7')(net['fconv6_BN'])
net['fconv7_7+']=Add()([net['fconv7_8to7_step2'],net['fconv7_7']]) # element-wise sum
net['fconv7_7+']=Activation('relu')(net['fconv7_7+'])
net['fconv7_7to1_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_7to1_step1')(net['fconv7_7+'])
net['fconv7_7to1_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_7to1_step2')(net['fconv7_7to1_step1'])

# ----------------------------------------- Structure 12-11-10-4 ------------------------------------------------

# fconv7_12, padding should be 3
weight_filter = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
net['fconv7_12']=Conv2D(128, (1,1), activation='relu', padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_12')(net['fconv6_BN'])
net['fconv7_12to11_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_12to11_step1')(net['fconv7_12'])
net['fconv7_12to11_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_12to11_step2')(net['fconv7_12to11_step1'])

# fconv7_11, padding should be 3
net['fconv7_11']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_11')(net['fconv6_BN'])
net['fconv7_11+']=Add()([net['fconv7_12to11_step2'],net['fconv7_11']]) # element-wise sum
net['fconv7_11+']=Activation('relu')(net['fconv7_11+'])
net['fconv7_11to10_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_11to10_step1')(net['fconv7_11+'])
net['fconv7_11to10_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_11to10_step2')(net['fconv7_11to10_step1'])

# fconv7_10, padding should be 3
net['fconv7_10']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_10')(net['fconv6_BN'])
net['fconv7_10+']=Add()([net['fconv7_11to10_step2'],net['fconv7_10']]) # element-wise sum
net['fconv7_10+']=Activation('relu')(net['fconv7_10+'])
net['fconv7_10to4_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_10to4_step1')(net['fconv7_10+'])
net['fconv7_10to4_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_10to4_step2')(net['fconv7_10to4_step1'])

print (net['fconv7_10to4_step2'])

# ---------------------------- Structure 2(elbow) and 7(hip) to 1(shoulder) --------------------------------
# fconv7_1, padding should be 3
net['fconv7_1']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1')(net['fconv6_BN'])
net['fconv7_1+']=Add()([net['fconv7_2to1_step2'], net['fconv7_7to1_step2'], net['fconv7_1']]) # element-wise sum
net['fconv7_1+']=Activation('relu')(net['fconv7_1+'])
net['fconv7_1to14_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1to14_step1')(net['fconv7_1+'])
net['fconv7_1to14_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1to14_step2')(net['fconv7_1to14_step1'])

print (net['fconv7_1to14_step2'])

# ---------------------------- Structure 5(elbow) and 10(hip) to 4(shoulder) -------------------------------
# fconv7_4, padding should be 3
net['fconv7_4']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4')(net['fconv6_BN'])
net['fconv7_4+']=Add()([net['fconv7_5to4_step2'], net['fconv7_10to4_step2'], net['fconv7_4']]) # element-wise sum
net['fconv7_4+']=Activation('relu')(net['fconv7_4+'])
net['fconv7_4to14_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4to14_step1')(net['fconv7_4+'])
net['fconv7_4to14_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4to14_step2')(net['fconv7_4to14_step1'])

print (net['fconv7_4to14_step2'])

# ---------------------------- Structure 4(shoulder) and 1(shoulder) to 14(neck) -------------------------------
# fconv7_14, padding should be 3
net['fconv7_14']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14')(net['fconv6_BN'])
net['fconv7_14+']=Add()([net['fconv7_1to14_step2'], net['fconv7_4to14_step2'], net['fconv7_14']]) # element-wise sum
net['fconv7_14+']=Activation('relu')(net['fconv7_14+'])
net['fconv7_14to13_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14to13_step1')(net['fconv7_14+'])
net['fconv7_14to13_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14to13_step2')(net['fconv7_14to13_step1'])

print (net['fconv7_14to13_step2'])

# -------------------------------------- Structure 14(neck) to 13(head top) -------------------------------
# fconv7_13, padding should be 3
net['fconv7_13']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_13')(net['fconv6_BN'])
net['pass_14to13']=Add()([net['fconv7_14to13_step2'], net['fconv7_13']]) # element-wise sum
net['fconv7_13+']=Activation('relu')(net['pass_14to13'])

print (net['fconv7_13+'])

Tensor("fconv7_2to1_step2_8/BiasAdd:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("fconv7_5to4_step2_12/BiasAdd:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("fconv7_10to4_step2_5/BiasAdd:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("fconv7_1to14_step2_9/BiasAdd:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("fconv7_4to14_step2_8/BiasAdd:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("fconv7_14to13_step2_11/BiasAdd:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("activation_227/Relu:0", shape=(?, 56, 56, 128), dtype=float32)


In [103]:
# --------------------------------- reverse of the structural tree ------------------------------------

# -------------------------------------- Structure 13(head top) to 14(neck) -------------------------------
net['fconvB7_13']=Conv2D(128, (1,1), activation='relu', padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_13')(net['fconv6_BN'])
net['fconv7_13to14_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_13to14_step1')(net['fconvB7_13'])
net['fconv7_13to14_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_13to14_step2')(net['fconv7_13to14_step1'])

# ---------------------------- Str-ucture 14(neck) to 1(shoulder) and 4(shoulder) --------------------------
net['fconvB7_14']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_14')(net['fconv6_BN'])
net['fconvB7_14+']=Add()([net['fconv7_13to14_step2'], net['fconvB7_14']]) # element-wise sum
net['fconvB7_14+']=Activation('relu')(net['fconvB7_14+'])
net['fconv7_14to1_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14to1_step1')(net['fconvB7_14+'])
net['fconv7_14to1_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14to1_step2')(net['fconv7_14to1_step1'])
net['fconv7_14to4_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14to4_step1')(net['fconvB7_14+'])
net['fconv7_14to4_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_14to4_step2')(net['fconv7_14to4_step1'])

# ---------------------------- Structure 1(shoulder) to 2(elbow) and 7(hip)
net['fconvB7_1']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_1')(net['fconv6_BN'])
net['fconvB7_1+']=Add()([net['fconv7_14to1_step2'], net['fconvB7_1']]) # element-wise sum
net['fconvB7_1+']=Activation('relu')(net['fconvB7_1+'])
net['fconv7_1to2_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1to2_step1')(net['fconvB7_1+'])
net['fconv7_1to2_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1to2_step2')(net['fconv7_1to2_step1'])
net['fconv7_1to7_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1to7_step1')(net['fconvB7_1+'])
net['fconv7_1to7_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_1to7_step2')(net['fconv7_1to7_step1'])

# ---------------------------- Structure 4(shoulder) to 5(elbow) and 10(hip)
net['fconvB7_4']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_4')(net['fconv6_BN'])
net['fconvB7_4+']=Add()([net['fconv7_14to4_step2'], net['fconvB7_4']]) # element-wise sum
net['fconvB7_4+']=Activation('relu')(net['fconvB7_4+'])
net['fconv7_4to5_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4to5_step1')(net['fconvB7_4+'])
net['fconv7_4to5_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4to5_step2')(net['fconv7_4to5_step1'])
net['fconv7_4to10_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4to10_step1')(net['fconvB7_4+'])
net['fconv7_4to10_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_4to10_step2')(net['fconv7_4to10_step1'])

# ------------------------------------ Structure 2(elbow) to 3(hand) -----------------------------------------
net['fconvB7_2']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_2')(net['fconv6_BN'])
net['fconvB7_2+']=Add()([net['fconv7_1to2_step2'], net['fconvB7_2']]) # element-wise sum
net['fconvB7_2+']=Activation('relu')(net['fconvB7_2+'])
net['fconv7_2to3_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_2to3_step1')(net['fconvB7_2+'])
net['fconv7_2to3_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_2to3_step2')(net['fconv7_2to3_step1'])

net['fconvB7_3']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_3')(net['fconv6_BN'])
net['fconvB7_3+']=Activation('relu')(Add()([net['fconv7_2to3_step2'], net['fconv7_3']]))

# ------------------------------------ Structure 5(elbow) to 6(hand) -----------------------------------------
net['fconvB7_5']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_5')(net['fconv6_BN'])
net['fconvB7_5+']=Add()([net['fconv7_4to5_step2'], net['fconvB7_5']]) # element-wise sum
net['fconvB7_5+']=Activation('relu')(net['fconvB7_5+'])
net['fconv7_5to6_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_5to6_step1')(net['fconvB7_5+'])
net['fconv7_5to6_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_5to6_step2')(net['fconv7_5to6_step1'])

net['fconvB7_6']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_6')(net['fconv6_BN'])
net['fconvB7_6+']=Activation('relu')(Add()([net['fconv7_5to6_step2'], net['fconv7_6']]))

# ------------------------------------ Structure 7(hip)-8(knee)-9(foot) -----------------------------------------
net['fconvB7_7']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_7')(net['fconv6_BN'])
net['fconvB7_7+']=Add()([net['fconv7_1to7_step2'], net['fconvB7_7']]) # element-wise sum
net['fconvB7_7+']=Activation('relu')(net['fconvB7_7+'])
net['fconv7_7to8_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_7to8_step1')(net['fconvB7_7+'])
net['fconv7_7to8_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_7to8_step2')(net['fconv7_7to8_step1'])

net['fconvB7_8']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_8')(net['fconv6_BN'])
net['fconvB7_8+']=Add()([net['fconv7_7to8_step2'], net['fconvB7_8']]) # element-wise sum
net['fconvB7_8+']=Activation('relu')(net['fconvB7_8+'])
net['fconv7_8to9_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_8to9_step1')(net['fconvB7_8+'])
net['fconv7_8to9_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_8to9_step2')(net['fconv7_8to9_step1'])

net['fconvB7_9']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_9')(net['fconv6_BN'])
net['fconvB7_9+']=Activation('relu')(Add()([net['fconv7_8to9_step2'], net['fconv7_9']]))

# ------------------------------------ Structure 10(hip)-11(knee)-12(foot) -----------------------------------------
net['fconvB7_10']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_10')(net['fconv6_BN'])
net['fconvB7_10+']=Add()([net['fconv7_4to10_step2'], net['fconvB7_10']]) # element-wise sum
net['fconvB7_10+']=Activation('relu')(net['fconvB7_10+'])
net['fconv7_10to11_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_10to11_step1')(net['fconvB7_10+'])
net['fconv7_10to11_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_10to11_step2')(net['fconv7_10to11_step1'])

net['fconvB7_11']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_11')(net['fconv6_BN'])
net['fconvB7_11+']=Add()([net['fconv7_10to11_step2'], net['fconvB7_11']]) # element-wise sum
net['fconvB7_11+']=Activation('relu')(net['fconvB7_11+'])
net['fconv7_11to12_step1']=Conv2D(64,(7,7), activation='relu', padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_11to12_step1')(net['fconvB7_11+'])
net['fconv7_11to12_step2']=Conv2D(128,(7,7), padding='same', use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconv7_11to12_step2')(net['fconv7_11to12_step1'])

net['fconvB7_12']=Conv2D(128, (1,1), padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_12')(net['fconv6_BN'])
net['fconvB7_12+']=Activation('relu')(Add()([net['fconv7_11to12_step2'], net['fconvB7_12']]))

print (net['fconvB7_3+'])
print (net['fconvB7_6+'])
print (net['fconvB7_9+'])
print (net['fconvB7_12+'])

Tensor("activation_232/Relu:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("activation_234/Relu:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("activation_237/Relu:0", shape=(?, 56, 56, 128), dtype=float32)
Tensor("activation_240/Relu:0", shape=(?, 56, 56, 128), dtype=float32)


In [104]:
net['fconvB7_27']=Conv2D(512, (1,1), activation='relu', padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter,name='fconvB7_27')(net['fconv6_BN'])
net['fconvB7_27+'] = Dropout(rate=0.5)(net['fconvB7_27'])

#-------------------------------------------- concate branches -------------------------------------------

net['fconv7_1_concat']=Concatenate(axis=3)([net['fconv7_1+'], net['fconvB7_1+']])
net['fconv7_2_concat']=Concatenate(axis=3)([net['fconv7_2+'], net['fconvB7_2+']])
net['fconv7_3_concat']=Concatenate(axis=3)([net['fconv7_3'], net['fconvB7_3+']])
net['fconv7_4_concat']=Concatenate(axis=3)([net['fconv7_4+'], net['fconvB7_4+']])
net['fconv7_5_concat']=Concatenate(axis=3)([net['fconv7_5+'], net['fconvB7_5+']])
net['fconv7_6_concat']=Concatenate(axis=3)([net['fconv7_6'], net['fconvB7_6+']])
net['fconv7_7_concat']=Concatenate(axis=3)([net['fconv7_7+'], net['fconvB7_7+']])
net['fconv7_8_concat']=Concatenate(axis=3)([net['fconv7_8+'], net['fconvB7_8+']])
net['fconv7_9_concat']=Concatenate(axis=3)([net['fconv7_9'], net['fconvB7_9+']])
net['fconv7_10_concat']=Concatenate(axis=3)([net['fconv7_10+'], net['fconvB7_10+']])
net['fconv7_11_concat']=Concatenate(axis=3)([net['fconv7_11+'], net['fconvB7_11+']])
net['fconv7_12_concat']=Concatenate(axis=3)([net['fconv7_12'], net['fconvB7_12+']])
net['fconv7_13_concat']=Concatenate(axis=3)([net['fconv7_13+'], net['fconvB7_13']])
net['fconv7_14_concat']=Concatenate(axis=3)([net['fconv7_14+'], net['fconvB7_14+']])
print (net['fconv7_14_concat'])

Tensor("concatenate_99/concat:0", shape=(?, 56, 56, 256), dtype=float32)


In [106]:
#-------------------------------------------- fconv9 -------------------------------------------
weight_filter_01 = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

net['fconv9_1']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_1')(net['fconv7_1_concat'])
net['fconv9_2']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_2')(net['fconv7_2_concat'])
net['fconv9_3']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_3')(net['fconv7_3_concat'])
net['fconv9_4']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_4')(net['fconv7_4_concat'])
net['fconv9_5']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_5')(net['fconv7_5_concat'])
net['fconv9_6']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_6')(net['fconv7_6_concat'])
net['fconv9_7']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_7')(net['fconv7_7_concat'])
net['fconv9_8']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_8')(net['fconv7_8_concat'])
net['fconv9_9']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_9')(net['fconv7_9_concat'])
net['fconv9_10']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_10')(net['fconv7_10_concat'])
net['fconv9_11']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_11')(net['fconv7_11_concat'])
net['fconv9_12']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_12')(net['fconv7_12_concat'])
net['fconv9_13']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_13')(net['fconv7_13_concat'])
net['fconv9_14']=Conv2D(13,(1,1),padding='valid',use_bias=True, bias_initializer='zeros', kernel_initializer=weight_filter_01,name='fconv9_14')(net['fconv7_14_concat'])

# ------------------------------------------ final concate --------------------------------------

net['fconv9'] = Concatenate(axis=3)([net['fconv9_1'],
net['fconv9_2'], net['fconv9_3'], net['fconv9_4'], net['fconv9_5'], net['fconv9_6'],
net['fconv9_7'], net['fconv9_8'], net['fconv9_9'], net['fconv9_10'], net['fconv9_11'],
net['fconv9_12'], net['fconv9_13'], net['fconv9_14']
])

print (net['fconv9'])


# generate the model
model = Model(net['input'], net['fconv9'])

print (model.layers)

Tensor("concatenate_101/concat:0", shape=(?, 56, 56, 182), dtype=float32)
[<keras.engine.topology.InputLayer object at 0x7ff4e51a4860>, <keras.layers.convolutional.Conv2D object at 0x7ff4e51a4898>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5203ba8>, <keras.layers.pooling.MaxPooling2D object at 0x7ff4e51a4978>, <keras.layers.convolutional.Conv2D object at 0x7ff4e514e2e8>, <keras.layers.convolutional.Conv2D object at 0x7ff4e51c4c18>, <keras.layers.pooling.MaxPooling2D object at 0x7ff4e514e9b0>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5182be0>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5171c18>, <keras.layers.convolutional.Conv2D object at 0x7ff4e5113f28>, <keras.layers.pooling.MaxPooling2D object at 0x7ff4e5122588>, <keras.layers.convolutional.Conv2D object at 0x7ff4e50e4710>, <keras.layers.convolutional.Conv2D object at 0x7ff4e51458d0>, <keras.layers.convolutional.Conv2D object at 0x7ff4e50e4550>, <keras.layers.convolutional.Conv2D object at 0x7ff4e50a9ac

In [111]:
def softmax_loss(self, y_true, y_pred):
        """Compute softmax loss.

        # Arguments
            y_true: Ground truth targets,
                tensor of shape (?, num_boxes, num_classes).
            y_pred: Predicted logits,
                tensor of shape (?, num_boxes, num_classes).

        # Returns
            softmax_loss: Softmax loss, tensor of shape (?, num_boxes).
        """
        y_pred = tf.maximum(tf.minimum(y_pred, 1 - 1e-15), 1e-15)
        softmax_loss = -tf.reduce_sum(y_true * tf.log(y_pred),
                                      axis=-1)
        return softmax_loss

In [113]:
# model.load_weights('''add weights''')
def schedule(epoch, decay=0.9):
    return base_lr * decay**(epoch)
callbacks = [keras.callbacks.ModelCheckpoint('./checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                                             verbose=1,
                                             save_weights_only=True),
             keras.callbacks.LearningRateScheduler(schedule)]

In [112]:
base_lr = 3e-4
optim = keras.optimizers.Adam(lr=base_lr)

model.compile(optimizer=optim,
              loss=softmax_loss)
num_epoch = 30
history = model.fit(self, x=None, y=None, 
                    batch_size=None, epochs=num_epoch, verbose=1, 
                    callbacks=None, 
                    validation_split=0.0, validation_data=None, 
                    shuffle=True, class_weight=None, sample_weight=None, 
                    initial_epoch=0, steps_per_epoch=None, 
                    validation_steps=None)
# history = model.fit_generator(gen.generate(True), gen.train_batches,
#                               num_epoch, verbose=1,
#                               callbacks=callbacks,
#                               validation_data=gen.generate(False),
#                               nb_val_samples=gen.val_batches,
#                               nb_worker=1)

TypeError: softmax_loss() missing 1 required positional argument: 'y_pred'